In [1]:
import pandas as pd
import numpy as np
%load_ext autoreload

# Load data

In [2]:
train_data = pd.read_csv('data/train_v2.csv')
columns = list(train_data)
columns

['id',
 'timestamp',
 'product_id',
 'product_department',
 'product_category',
 'card_id',
 'user_id',
 'C15',
 'C16',
 'C17',
 'C18',
 'C19',
 'C20',
 'C21',
 'amount',
 'isfraud']

# Train/Test split
* Normally I would use random sampling, stratified by an attribute of major relevance, however, in this case the test data that was given follows the train data in time. Therefore, in order to do local testing my first guess would be that it is better to remake that scenario and sample the data by simply splitting it sorted as it is, by time.
* Cross validation is not necessary given that we have a test set big enough

In [33]:
N_original, M_original = train_data.shape
split_by = 2
N_train = 100000
start_at = N_original - N_train * split_by
split_at = start_at + N_train
train_sample = pd.DataFrame(train_data.loc[start_at:split_at-1,:])
test_sample = pd.DataFrame(train_data.loc[split_at:,:])
train_sample.shape, test_sample.shape

((100000, 16), (100000, 16))

# Data pipeline
* With trees, normalizing features is not necessary

In [67]:
%autoreload
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from include.DatetimeFromTimestamp import DatetimeFromTimestamp
from include.DataFrameDropper import DataFrameDropper
from include.DataFrameSelector import DataFrameSelector
from include.FilterNMostCommon import FilterNMostCommon

columns_to_drop = ['id', 'timestamp', 'product_id', 'product_department', 'card_id', 'user_id']
#['product_department', 'product_category', 'C15', 'C16', 'C17', 'C18', 'C19', 'C20', 'amount', 'isfraud']

pipeline_normal = Pipeline([
    ('dataframe_dropper', DataFrameDropper(attribute_names='product_category')),
    ('datetime_creator', DatetimeFromTimestamp()),
])

pipeline_1hot = Pipeline([
    ('dataframe_selector', DataFrameSelector(['product_category'])),
    ('filter_n_most_common', FilterNMostCommon(N=5, attribute_name='product_category')),
    ('1hot_encoder', OneHotEncoder(sparse = False))
])

pipeline_full = FeatureUnion(transformer_list=[
    ('pipeline_normal', pipeline_normal),
    ('pipeline_1hot', pipeline_1hot),
])

#treated_data = DatetimeFromTimestamp().fit_transform(train_data)
#treated_data = DataFrameDropper(columns_to_drop).fit_transform(treated_data)

treated_data = pipeline_full.fit_transform(train_sample)

treated_data.shape

(100000, 22)

# Test

In [ ]:
from sklearn.tree import DecisionTreeClassifier

model = DecisionTreeClassifier()
model.fit(train_treated[:,:-1], train_treated[:,-1])

# Submit

In [8]:
vc = train_sample['product_category'].value_counts()
vc

50e219e0    44810
f028772b    29437
28905ebd    17342
3e814130     4551
75fa27f6     1554
76b2941d     1207
335d28a8      538
f66779e6      525
c0dd3be3       16
0569f928        7
a818d37a        5
70fb0e29        3
bcf865d9        3
dedf689d        1
72722551        1
Name: product_category, dtype: int64

In [31]:
%autoreload

filter_n_most_common = FilterNMostCommon(N=3, attribute_name='product_category')
filter_n_most_common.fit_transform(train_sample)

,id,timestamp,product_id,product_department,product_category,card_id,user_id,C15,C16,C17,C18,C19,C20,C21,amount,isfraud
32169524,32169524,1414536853630,c4e18dd6,85f751fd,50e219e0,9c13b419,a99f214a,320,50,2060,3,39,-1,23,191.77,0
32169525,32169525,1414534658825,c4e18dd6,85f751fd,50e219e0,51cedd4e,a99f214a,320,50,2480,3,297,100111,61,191.77,0
32169526,32169526,1414534850086,c4e18dd6,85f751fd,50e219e0,febd1138,a99f214a,320,50,2480,3,297,100111,61,191.77,0
32169527,32169527,1414534149689,c4e18dd6,85f751fd,50e219e0,3e2bf98d,58132d0c,320,50,2647,2,35,100148,23,191.77,0
32169528,32169528,1414536242057,98572c79,d9750ee7,f028772b,ecad2386,a99f214a,320,50,1993,2,1063,100083,33,177.65,1
32169529,32169529,1414536360515,c4e18dd6,85f751fd,50e219e0,92f5800b,a99f214a,320,50,2424,1,161,100189,71,191.77,0
32169530,32169530,1414536948929,f3845767,1fbe01fe,28905ebd,ecad2386,a99f214a,320,50,2617,0,35,100084,51,184.09,0
32169531,32169531,1414535375498,5c9ae867,83a0ad1a,f028772b,ecad2386,a99f214a,320,50,2665,0,35,100193,221,198.20,0
32169532,32169532,1414534875452,7e091613,e151e245,f028772b,ecad2386,a99f214a,320,50,1872,3,39,-1,23,169.51,1
32169533,32169533,1414534427141,7e091613,e151e245,f028772b,ecad2386,a99f214a,320,50,2647,2,35,100148,23,169.51,0


In [ ]:
Data